In [1]:
# Import libraries
#!pip install openpyxl --upgrade
import pandas as pd
import re
import ollama
import numpy as np

In [2]:
# Read case summaries Excel file and create a DataFrame
df = pd.read_excel("case_summarization.xlsx")  
df['Body']=df['Body'].apply(str)

df.head()

,Case: Case Number,Case: Subject,Case: Description,Case: Product Name,Automation Item Name,Created Date,Body,Feedback,Detailed Feedback,Case: Status,Case: Client Escalated
0,TS017493063,*JLO* GBM-Kuwait-KOC-ESS- PROACTIVE ACTION AGA...,I am reaching out to discuss the IBM support a...,Elastic Storage System Hardware,AI-1559438,2024-10-13,Customer Sentiment negative Description Summar...,None,NaN,IBM is working,0
1,TS017525407,Email - We required high uninitialized Luns/ho...,We required high uninitialized Luns/hosts (Hig...,DS8900F,AI-1560218,2024-10-14,Customer Sentiment Not available Description S...,None,good,Closed by IBM,0
2,TS017529903,fca72a160 7/12 -> fca72a163 7/8 QSFP port do n...,fca72a160 7/12 -> fca72a163 7/8 QSFP port do n...,SAN b-type Collection,AI-1560770,2024-10-14,Customer Sentiment neutral Description Summary...,NaN,NaN,Closed by Client,0
3,TS014486964,SSD problem,Please note that there is a problem with one o...,Power System S914 Server,AI-1560820,2024-10-14,Customer Sentiment neutral Description Summary...,NaN,NaN,Awaiting your feedback,0
4,TS017567622,Blade unexpectedly powered off. No entries in ...,Synergy blade ohecp1esxi002 unexpectedly power...,Synergy Series Frames,AI-1561315,2024-10-14,Customer Sentiment negative Description Summar...,NaN,NaN,Closed by IBM,0


In [3]:
# 1.Remove duplicates
clean_data = df.drop_duplicates()

In [4]:
# 2.Handle missing values
clean_data = clean_data.dropna()


In [5]:
# Tokenization and preprocessing

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(Body):
 tokens = word_tokenize(Body)
 tokens = [token.lower() for token in tokens if token.isalpha()]
 tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
 return ' '.join(tokens)

#Read the original CSV file
df = pd.read_excel("case_summarization.xlsx")

#  Apply the preprocess_text function to the 'Body' column
clean_data['processed_text'] = clean_data['Body'].apply(preprocess_text)

# Write the cleaned DataFrame with the new column to a new CSV file
clean_data.to_csv('clean_data_with_processed_text.csv', index=False)



In [6]:
# Convert 'Created Date', 'open_date', and 'close_date' to datetime format
clean_data['Created Date'] = pd.to_datetime(clean_data['Created Date'])
#clean_data['open_date'] = pd.to_datetime(clean_data['open_date'])
clean_data['Close_date'] = pd.to_datetime(2024-10-10)

# Calculate the difference in days between 'close_date' and 'Created Date is time_taken '
clean_data['days_difference'] = (clean_data['Close_date'] - clean_data['Created Date']).dt.days

# Step 4: Print the updated DataFrame
print(clean_data[['Created Date','days_difference' ]])

    Created Date  days_difference
1     2024-10-14           -20010
9     2024-10-14           -20010
11    2024-10-14           -20010
13    2024-10-14           -20010
15    2024-10-14           -20010
..           ...              ...
411   2024-10-18           -20014
425   2024-10-18           -20014
426   2024-10-18           -20014
427   2024-10-18           -20014
434   2024-10-19           -20015

[61 rows x 2 columns]


In [8]:
# Read cleam data in processed_text and create a DataFrame
df = pd.read_csv("clean_data_with_processed_text.csv")  
df['processed_text']=df['processed_text'].apply(str)

df.head()

,Case: Case Number,Case: Subject,Case: Description,Case: Product Name,Automation Item Name,Created Date,Body,Feedback,Detailed Feedback,Case: Status,Case: Client Escalated,processed_text
0,TS017525407,Email - We required high uninitialized Luns/ho...,We required high uninitialized Luns/hosts (Hig...,DS8900F,AI-1560218,2024-10-14,Customer Sentiment Not available Description S...,None,good,Closed by IBM,0,customer sentiment available description summa...
1,TS017538910,SWITCH REBOOT,"2 reboot events ""Non restartable component (fd...",SAN b-type Collection,AI-1561615,2024-10-14,Customer Sentiment neutral Description Summary...,None,ok,Closed by IBM,0,customer sentiment neutral description summary...
2,TS017505135,Email-LUN Total Latency is greater than or equ...,LUN Total Latency is greater than or equal to ...,FlashSystem 9200,AI-1562015,2024-10-14,Customer Sentiment Not available Description S...,None,"Very useful feather to have , thanks",IBM is working,0,customer sentiment available description summa...
3,TS017519459,Expired Certificates (EXPIRED_CERTS),"Good afternoon, we recently received several w...",SAN b-type Collection,AI-1562168,2024-10-14,Customer Sentiment neutral Description Summary...,None,ok,Closed by IBM,0,customer sentiment neutral description summary...
4,TS017504642,Email - Need to change the SFP fault,Need to change the SFP,SAN c-type Collection,AI-1562711,2024-10-14,Customer Sentiment neutral Description Summary...,None,ok,Closed by IBM,0,customer sentiment neutral description summary...
